# Heated Wall in Simple configuration - Verification sheet

Verification Test for the XNSFE_Solver,  
The Validation Test Runner is running these via `jupyter nbconvert` with _'hold on error'_ so if the assertion in the end fails the whole job fails.

Setup:
  * Interface at 90°.  
  * Equal fluid densities => simplified setting  
  * Also no Heat capacity => infinitely fast heat conduction  
  * Reference Simulation done with FastMarching

## Load BoSSS

In [ ]:
// #r "../../../src/L4-application/BoSSSpad/bin/Release/net5.0/BoSSSpad.dll"
// gets copied to ValidationTestRunner
#r "BoSSSpad.dll"
using System;
using System.Collections.Generic;
using System.Linq;
using ilPSP;
using ilPSP.Utils;
using BoSSS.Platform;
using BoSSS.Foundation;
using BoSSS.Foundation.XDG;
using BoSSS.Foundation.Grid;
using BoSSS.Foundation.Grid.Classic;
using BoSSS.Foundation.IO;
using BoSSS.Solution;
using BoSSS.Solution.Control;
using BoSSS.Solution.GridImport;
using BoSSS.Solution.Statistic;
using BoSSS.Solution.Utils;
using BoSSS.Solution.AdvancedSolvers;
using BoSSS.Solution.Gnuplot;
using BoSSS.Application.BoSSSpad;
using BoSSS.Application.XNSE_Solver;
using static BoSSS.Application.BoSSSpad.BoSSSshell;
Init();

## Setup Workflowmanagement, Batchprocessor and Database

In [ ]:
ExecutionQueues

index,type,DeploymentBaseDirectory,DeployRuntime,Name,DotnetRuntime,Username,ServerName,ComputeNodes,DefaultJobPriority,SingleNode,AllowedDatabasesPaths,BatchInstructionDir
0,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,<null>,dotnet,FDY\rieckmann,DC2,<null>,Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann, PathAtRemote: } ]",
1,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER,dotnet,FDY\rieckmann,DC2,[ hpccluster ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
2,BoSSS.Application.BoSSSpad.MsHPC2012Client,\\hpccluster\hpccluster-scratch\rieckmann\binaries,True,HPCCLUSTER2,dotnet,FDY\rieckmann,DC2,[ hpccluster2 ],Normal,True,"[ { \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB == : LocalMountPath: \\hpccluster\hpccluster-scratch\rieckmann\BoSSS_DB, PathAtRemote: } ]",
3,BoSSS.Application.BoSSSpad.MiniBatchProcessorClient,C:\Users\rieckmann\AppData\Local\BoSSS-LocalJobs,False,<null>,dotnet,,,,,,,<null>


In [ ]:
static var myBatch = BoSSSshell.GetDefaultQueue();

In [ ]:
string ProjectName = $"HeatedWallSimple_VerificationFastMarching";

In [ ]:
BoSSSshell.WorkflowMgm.Init(ProjectName);

Project name is set to 'HeatedWallSimple_VerificationFastMarching_2021111'.
Creating database '\\hpccluster\hpccluster-scratch\rieckmann\HeatedWallSimple_VerificationFastMarching_2021111'.


In [ ]:
static var myDb = BoSSSshell.WorkflowMgm.DefaultDatabase;

## Setup Simulationcontrols

In [ ]:
using BoSSS.Application.XNSFE_Solver;

In [ ]:
int[] hRes = {4};//{1, 2, 3, 4};
int[] pDeg = {2};//{ 1, 2, 3, 4};
double[] Q = {0.0, 0.05, 0.1, 0.2};

In [ ]:
List<XNSFE_Control> Controls = new List<XNSFE_Control>();
foreach(int h in hRes){
    foreach(int p in pDeg){
        foreach(double q in Q){

        var ctrl = new XNSFE_Control();
        ctrl.Paramstudy_CaseIdentification.Add(new Tuple<string, object>("HeatFlux", q));

        ctrl.DbPath      = null;
        ctrl.SessionName = $"HeatedWall_Simple_res:{h}_p:{p}_Q:{q}";
        ctrl.ProjectName = $"ProjectName";
        ctrl.SetDatabase(myDb);
        ctrl.savetodb = true;        

        ctrl.SetDGdegree(p);

        // For Gravity the above routine does not contain standard values, add the options manually
        ctrl.FieldOptions.Add("GravityX#A", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });
        ctrl.FieldOptions.Add("GravityY#A", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });
        ctrl.FieldOptions.Add("GravityX#B", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });
        ctrl.FieldOptions.Add("GravityY#B", new FieldOpts() {
            SaveToDB = FieldOpts.SaveToDBOpt.TRUE
        });        

        #region grid
        double L = 5.0;
        int kelemR = h;
        string[] Bndy = new string[] {  "Inner",
                                        "NavierSlip_linear_ConstantHeatFlux_right",
                                        "pressure_outlet_ZeroGradient_top",
                                        "freeslip_ZeroGradient_left",
                                        "pressure_outlet_ZeroGradient_bottom"};

        ctrl.GridFunc = delegate () {
            double[] Xnodes = GenericBlas.Linspace(-L, 0, kelemR + 1);
            double[] Ynodes = GenericBlas.Linspace(0, 3 * L, 3 * kelemR + 1);
            var grd = Grid2D.Cartesian2DGrid(Xnodes, Ynodes);

            for(byte i= 1; i < Bndy.Count(); i++) {
                grd.EdgeTagNames.Add(i, Bndy[i]);
            }

            grd.DefineEdgeTags(delegate (double[] X) {
                byte et = 0;
                if(Math.Abs(X[0] - Xnodes.Last()) < 1e-8)
                    return 1;
                if(Math.Abs(X[0] - Xnodes.First()) < 1e-8)
                    return 3;
                if(Math.Abs(X[1] - Ynodes.Last()) < 1e-8)
                    return 2;
                if(Math.Abs(X[1] - Ynodes.First()) < 1e-8)
                    return 4;
                return et;
            });

            return grd;
        };
        #endregion

        #region material
        ctrl.PhysicalParameters = new BoSSS.Solution.XNSECommon.PhysicalParameters() {
            rho_A = 1.0, // 958.0
            rho_B = 1.0, // 0.59,

            mu_A = 1, //2.82 * 1e-4,
            mu_B = 0.001, //1.23 * 1e-6,

            Sigma = 1.0,
            betaS_A = 1000, // sliplength is mu/beta
            betaS_B = 1000,
        };

        ctrl.ThermalParameters = new BoSSS.Solution.XheatCommon.ThermalParameters() {
            rho_A = 1.0, // 958.0
            rho_B = 1.0, //0.59,

            k_A = 1.0, // 0.6
            k_B = 1.0, // 0.026,

            c_A = 0.0,
            c_B = 0.0,

            hVap = 1,//2.257 * 1e6,
            T_sat = 0.0 // 373.0
        };

        ctrl.PhysicalParameters.IncludeConvection = true;
        ctrl.ThermalParameters.IncludeConvection = true;
        ctrl.PhysicalParameters.Material = false;
        #endregion

        #region Initial Condition - Exact Solution

        // solution for massflux and velocity at level set
        double y0 = 0.2 * L;

        // inital values
        double g = 4;
        ctrl.AddInitialValue("Phi", $"(X, t) => -{y0} + X[1]", true);
        ctrl.AddInitialValue("Temperature#A", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
        ctrl.AddInitialValue("Temperature#B", $"(X, t) => {ctrl.ThermalParameters.T_sat}", true);
        ctrl.AddInitialValue("GravityY#A", $"(X, t) => -{g}", true);

        #endregion

        #region Boundary Conditions

        double v = 1.0;
        ctrl.AddBoundaryValue(Bndy[1], "HeatFluxX#A", $"(X, t) => {q}", true);
        ctrl.AddBoundaryValue(Bndy[1], "VelocityY#A", $"(X, t) => {v}", true);
        ctrl.AddBoundaryValue(Bndy[1], "VelocityY#B", $"(X, t) => {v}", true);


        ctrl.AddBoundaryValue(Bndy[3]);
        ctrl.AddBoundaryValue(Bndy[2]);
        ctrl.AddBoundaryValue(Bndy[4], "Pressure#A", $"(X, t) => {y0} * {ctrl.PhysicalParameters.rho_A} * {g}", true);

        #endregion

        #region AMR

        int level = 3;
        ctrl.AdaptiveMeshRefinement = level > 0;
        ctrl.activeAMRlevelIndicators.Add(new BoSSS.Solution.LevelSetTools.SolverWithLevelSetUpdater.AMRonNarrowband() { maxRefinementLevel = level });
        ctrl.AMR_startUpSweeps = level;

        #endregion

        #region Timestepping

        ctrl.AdvancedDiscretizationOptions.SST_isotropicMode = BoSSS.Solution.XNSECommon.SurfaceStressTensor_IsotropicMode.LaplaceBeltrami_ContactLine;
        ctrl.Option_LevelSetEvolution = BoSSS.Solution.LevelSetTools.LevelSetEvolution.FastMarching;
        ctrl.Timestepper_LevelSetHandling =  BoSSS.Solution.XdgTimestepping.LevelSetHandling.LieSplitting;

        ctrl.NonLinearSolver.SolverCode = NonLinearSolverCode.Newton;
        ctrl.NonLinearSolver.Globalization = BoSSS.Solution.AdvancedSolvers.Newton.GlobalizationOption.Dogleg;
        ctrl.NonLinearSolver.ConvergenceCriterion = 1e-8;
        ctrl.NonLinearSolver.MaxSolverIterations = 10;

        ctrl.SkipSolveAndEvaluateResidual = false;

        ctrl.TimeSteppingScheme = BoSSS.Solution.XdgTimestepping.TimeSteppingScheme.ImplicitEuler;
        ctrl.TimesteppingMode = BoSSS.Solution.Control.AppControl._TimesteppingMode.Transient;
        ctrl.dtFixed = 0.01;
        ctrl.Endtime = 15.0;
        ctrl.NoOfTimesteps = (int)(ctrl.Endtime / ctrl.dtFixed);

        #endregion
        ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MassfluxLogging() { LogPeriod = 1 });
        ctrl.PostprocessingModules.Add(new BoSSS.Application.XNSFE_Solver.PhysicalBasedTestcases.MovingContactLineLogging() { LogPeriod = 1 });

        Controls.Add(ctrl);
        }
    }
}

In [ ]:
System.Diagnostics.Process.GetCurrentProcess().HandleCount

In [ ]:
Controls.Count

## Start simulations on Batch processor

In [ ]:
foreach(var C in Controls) {
    Type solver = typeof(BoSSS.Application.XNSFE_Solver.XNSFE<XNSFE_Control>);

    string jobName       = C.SessionName;
    var oneJob           = new Job(jobName, solver);
    oneJob.NumberOfMPIProcs = 1;    
    oneJob.SetControlObject(C);
    oneJob.Activate(myBatch, false);
}

### Wait until jobs complete

In [ ]:
BoSSSshell.WorkflowMgm.BlockUntilAllJobsTerminate(86400 , 60); // wait for one day, or until all jobs finish

## Postprocessing

Track the amount of errors occuring

In [ ]:
int Errors = 0;

### Load sessions

In [ ]:
var sessions = BoSSSshell.WorkflowMgm.Sessions.Where(s => s.SuccessfulTermination == true).ToList();

In [ ]:
sessions

#0: HeatedWallSimple_VerificationFastMarching_2021111	HeatedWall_Simple_res:4_p:2_Q:0	11/01/2021 16:29:40	10160514...
#1: HeatedWallSimple_VerificationFastMarching_2021111	HeatedWall_Simple_res:4_p:2_Q:0.05	11/01/2021 16:30:16	c7ebb8c1...
#2: HeatedWallSimple_VerificationFastMarching_2021111	HeatedWall_Simple_res:4_p:2_Q:0.1	11/01/2021 16:30:16	18a766e0...
#3: HeatedWallSimple_VerificationFastMarching_2021111	HeatedWall_Simple_res:4_p:2_Q:0.2	11/01/2021 16:30:44	c64fb30a...


In [ ]:
if(sessions.Count < 4){
    Console.WriteLine("\x1b[31mError: expected 4 successful sessions!\x1b[0m");
    Errors += 4 - sessions.Count;
}

#### Load data

In [ ]:
List<Plot2Ddata> dataCL, dataEvap;

try{
    dataCL = sessions.ReadLogDataForMovingContactLine()[1];
    dataEvap = sessions.ReadLogDataForXNSE("Massflux");
} catch {
    Console.WriteLine("\x1b[31mError: loading data!\x1b[0m");
    Errors++;
}

number of contact lines: 2
Element at 0: time vs contact-pointX
Element at 1: time vs contact-pointY
Element at 2: time vs contact-VelocityX
Element at 3: time vs contact-VelocityY
Element at 4: time vs contact-angle
Element at 0: time vs mass-liq
Element at 1: time vs mass-vap
Element at 2: time vs mass-total
Element at 3: time vs masschange-evap
Element at 4: time vs masschange-vapor
Element at 5: time vs masschange-liquid
Element at 6: time vs masschange-total
Element at 7: time vs interface length


### Process data

In [ ]:
try{
    dataCL.Pick(4).dataGroups.ForEach(s => s.Values = s.Values.Select(v => 180 - v ).ToArray());

    Plot2Ddata evapFluxes = new Plot2Ddata();
    evapFluxes.dataGroups = new Plot2Ddata.XYvalues[dataEvap.Pick(3).dataGroups.Length]; // same as total evaporative heatflux * dt, as rho = hvap = 1
    Plot2Ddata bndFluxes = new Plot2Ddata();
    bndFluxes.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(1).dataGroups.Length]; 
    Plot2Ddata.XYvalues[] Vals = new Plot2Ddata.XYvalues[dataCL.Pick(4).dataGroups.Length];
    Plot2Ddata clVeloY = new Plot2Ddata();
    clVeloY.dataGroups = new Plot2Ddata.XYvalues[dataCL.Pick(3).dataGroups.Length]; 
    int dashType = 0;
    for(int i = 0; i<sessions.Count;i++){
        sessions[i].KeysAndQueries.TryGetValue("id:PressureRatio", out object color);
        color = color == null ? 7 : Convert.ToInt32(color);
        dataCL.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
        dataEvap.ForEach(p => p.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]));
        PlotFormat PF = new PlotFormat(lineColor: (LineColors)color, dashType: ((DashTypes)(++dashType)));
        PF.LineWidth = 3.0;
        dataCL.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
        dataEvap.ForEach(p => p.dataGroups[i].Format = PF.CloneAs());
        
        double alpha = 180/Math.PI * 0.5*Math.Asin(2*Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"]));
        Vals[i] = new Plot2Ddata.XYvalues("q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]), new double[] {0.0, 15}, new double[] {alpha, alpha});
        Vals[i].Format = new PlotFormat("b-");

        double dt = Convert.ToDouble(sessions[i].KeysAndQueries["dtFixed"]);
        PF.LineWidth = 3.0;
        PF.LineColor = (LineColors)7;
        PF.DashType = (DashTypes)1;
        evapFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
        evapFluxes.dataGroups[i].Values = dataEvap.Pick(3).dataGroups[i].Values.Select(x => x).ToArray();
        evapFluxes.dataGroups[i].Abscissas = dataEvap.Pick(3).dataGroups[i].Abscissas.Select(x => x).ToArray();
        evapFluxes.dataGroups[i].Name = dataEvap.Pick(3).dataGroups[i].Name;
        evapFluxes.dataGroups[i].Format = PF.CloneAs();
        evapFluxes.dataGroups[i].Values = evapFluxes.dataGroups[i].Values.Select(v => v / dt).ToArray();

        PF.LineWidth = 3.0;
        PF.LineColor = (LineColors)2;
        PF.DashType = (DashTypes)2;
        bndFluxes.dataGroups[i] = new Plot2Ddata.XYvalues("");
        bndFluxes.dataGroups[i].Values = dataCL.Pick(1).dataGroups[i].Values.Select(x => x).ToArray();
        bndFluxes.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
        bndFluxes.dataGroups[i].Name = "Q-Bnd:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
        bndFluxes.dataGroups[i].Format = PF.CloneAs();
        bndFluxes.dataGroups[i].Values = bndFluxes.dataGroups[i].Values.Select(v => v * Convert.ToDouble(sessions[i].KeysAndQueries["id:HeatFlux"])).ToArray();

        PF.LineColor = (LineColors)color;
        PF.DashType = (DashTypes)dashType;
        clVeloY.dataGroups[i] = new Plot2Ddata.XYvalues("");
        clVeloY.dataGroups[i].Values = new double[dataCL.Pick(3).dataGroups[i].Values.Length];
        for(int j = 0; j<dataCL.Pick(3).dataGroups[i].Values.Length; j++){
            clVeloY.dataGroups[i].Values[j] = dataCL.Pick(3).dataGroups[i].Values[j] - dataCL.Pick(2).dataGroups[i].Values[j] / Math.Tan(Math.PI / 180.0 * dataCL.Pick(4).dataGroups[i].Values[j]);
        } 
        clVeloY.dataGroups[i].Abscissas = dataCL.Pick(1).dataGroups[i].Abscissas.Select(x => x).ToArray();
        clVeloY.dataGroups[i].Name = "q:" + Convert.ToString(sessions[i].KeysAndQueries["id:HeatFlux"]);
        clVeloY.dataGroups[i].Format = PF.CloneAs();
    }
    dataCL.Pick(4).dataGroups = dataCL.Pick(4).dataGroups.ToList().Cat(Vals).ToArray();    
    evapFluxes = evapFluxes.Merge(bndFluxes);
    dataEvap[3] = evapFluxes; // reassign...
    dataCL[3] = clVeloY;
} catch {
    Console.WriteLine("\x1b[31mError: processing data!\x1b[0m");
    Errors++;
}

### Comparison to reference data

#### (De)Serialization of reference values

In [ ]:
using System.IO;
using Newtonsoft.Json;

In [ ]:
JsonSerializer formatter = new JsonSerializer() {
    NullValueHandling = NullValueHandling.Ignore,
    TypeNameHandling = TypeNameHandling.Auto,
    ConstructorHandling = ConstructorHandling.AllowNonPublicDefaultConstructor,
    ReferenceLoopHandling = ReferenceLoopHandling.Error,
    Formatting = Formatting.Indented
};            

//originally called to write out reference data
// using(var tw = new StringWriter()) {
//     using(JsonWriter writer = new JsonTextWriter(tw)) {
//         formatter.Serialize(writer, dataCL);
//         string Ret = tw.ToString(); 
//         File.WriteAllText(".\\ReferenceData\\dataCL_Reference.json", Ret); 
//     }
// }  
// using(var tw = new StringWriter()) {
//     using(JsonWriter writer = new JsonTextWriter(tw)) {          
//         formatter.Serialize(writer, dataEvap);
//         string Ret = tw.ToString(); 
//         File.WriteAllText(".\\ReferenceData\\dataEvap_Reference.json", Ret);   
//     }    
// }

In [ ]:
List<Plot2Ddata> dataCL_Ref, dataEvap_Ref;

using(var tr = new StringReader(File.ReadAllText(".\\dataCL_Reference.json"))) {        
    using(JsonReader reader = new JsonTextReader(tr)) {
            var obj = formatter.Deserialize(reader, typeof(List<Plot2Ddata>));
            dataCL_Ref = (List<Plot2Ddata>)obj;
        }         
}
using(var tr = new StringReader(File.ReadAllText(".\\dataEvap_Reference.json"))) {        
    using(JsonReader reader = new JsonTextReader(tr)) {
            var obj = formatter.Deserialize(reader, typeof(List<Plot2Ddata>));
            dataEvap_Ref = (List<Plot2Ddata>)obj;
        }         
}

#### Setup Plot comparison

Left - current data  
Right - reference data

In [ ]:
var gp = new Gnuplot();
gp.SetMultiplot(4, 2);
Plot2Ddata data;

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [ ]:
try{
    gp.SetSubPlot(0, 0);

    data = dataCL.Pick(4);
    data.Xlabel = "time [s]";
    data.Ylabel = "angle [°]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact Angle";

    data.ToGnuplot(gp);

    gp.SetSubPlot(0, 1);

    data = dataCL_Ref.Pick(4);
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataCL.Pick(4).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "angle [°]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact Angle Reference";

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

set key font ",16"Left reverse 
set key font ",16"Left reverse 


In [ ]:
try{
    gp.SetSubPlot(1, 0);

    data = dataEvap.Pick(3);
    data.Xlabel = "time [s]";
    data.Ylabel = "Total Heat Flux [W]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Heatflux";
    data.ShowLegend = false;

    data.ToGnuplot(gp);

    gp.SetSubPlot(1, 1);

    data = dataEvap_Ref.Pick(3);
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataEvap.Pick(3).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "Total Heat Flux [W]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Heatflux Reference";
    data.ShowLegend = false;

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

In [ ]:
try{
    gp.SetSubPlot(2, 0);

    data = dataCL.Pick(1);
    data.Xlabel = "time [s]";
    data.Ylabel = "Pos Y [m]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Position";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(2, 1);

    data = dataCL_Ref.Pick(1);
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataCL.Pick(1).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "Pos Y [m]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Position Reference";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

set key font ",16"inside left top Left reverse 
set key font ",16"inside left top Left reverse 


In [ ]:
try{
    gp.SetSubPlot(3, 0);

    data = new Plot2Ddata();
    // Filter the velocity for a smoother representation
    foreach(var group in dataCL.Pick(3).dataGroups){
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        double n, m;
        for(int i = 1; i< val.Values.Count()-1; i++){
            m = val.Values[i + 1];
            n = val.Values[i - 1];
            double mid = 0.5 * (n+m);        
            if(Math.Abs(val.Values[i] - n) > Math.Abs(m - n) )
                val.Values[i] = mid;
        }
        val.Format = group.Format;
        data.dataGroups = data.dataGroups.Cat(val);
    }
    data.Xlabel = "time [s]";
    data.Ylabel = "V_y [m/s]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Velocity";
    data.LegendAlignment = new string[] {"i","l","b"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(3, 1);

    data = new Plot2Ddata();
    // Filter the velocity for a smoother representation
    foreach(var group in dataCL_Ref.Pick(3).dataGroups){
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        double n, m;
        for(int i = 1; i< val.Values.Count()-1; i++){
            m = val.Values[i + 1];
            n = val.Values[i - 1];
            double mid = 0.5 * (n+m);        
            if(Math.Abs(val.Values[i] - n) > Math.Abs(m - n) )
                val.Values[i] = mid;
        }
        val.Format = group.Format;
        data.dataGroups = data.dataGroups.Cat(val);
    }
    for(int i=0; i<data.dataGroups.Count(); i++) { data.dataGroups[i].Format = dataCL.Pick(3).dataGroups[i].Format; }
    data.Xlabel = "time [s]";
    data.Ylabel = "V_y [m/s]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Velocity Reference";
    data.LegendAlignment = new string[] {"i","l","b"};

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

set key font ",16"inside left bottom Left reverse 
set key font ",16"inside left bottom Left reverse 


#### Plot comparison

In [ ]:
gp.PlotSVG(1600, 4 * 600)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 0 
 
 
 
 
 10 
 
 
 
 
 20 
 
 
 
 
 30 
 
 
 
 
 40 
 
 
 
 
 50 
 
 
 
 
 60 
 
 
 
 
 70 
 
 
 
 
 80 
 
 
 
 
 90 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 angle [°] 
 
 
 
 
 time [s] 
 
 
 
 
 Contact Angle 
 
 
 q:0 
 
 
 
 
 q:0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M602.7,117.1 L656.1,117.1 M71.5,96.1 L72.1,96.1 L72.6,120.6 L73.2,137.0 L73.7,150.2 L74.3,161.3
 L74.9,171.3 L75.4,179.9 L76.0,187.8 L76.5,194.9 L77.1,201.3 L77.6,205.9 L78.2,209.7 L78.8,216.3
 L79.3,221.7 L79.9,226.7 L80.4,231.3 L81.0,235.8 L81.6,239.9 L82.1,243.6 L82.7,242.3 L83.2,247.5
 L83.8,251.9 L84.4,255.9 L84.9,259.6 L85.5,263.1 L86.0,266.7 L86.6,268.9 L87.1,263.5 L87.7,268.4
 L88.3,268.9 L88.8,273.9 L89.4,277.7 L89.9,281.0 L90.5,284.0 L91.1,287.0 L91.6,290.0 L92.2,292.8
 L92.7,295.4 L93.3,298.0 L93.9,300.5 L94.4,299.2 L95.0,302.9 L95.5,307.0 L96.1,301.9 L96.6,305.3
 L97.2,307.3 L97.8,309.0 L98.3,310.6 L98.9,312.3 L99.4,314.1 L100.0,316.0 L100.6,317.7 L101.1,319.5
 L101.7,318.5 L102.2,320.7 L102.8,323.7 L103.4,326.3 L103.9,328.7 L104.5,334.3 L105.0,327.1 L105.6,326.8
 L106.1,329.6 L106.7,331.1 L107.3,332.6 L107.8,334.1 L108.4,335.8 L108.9,337.5 L109.5,339.1 L110.1,340.7
 L110.6,342.3 L111.2,340.4 L111.7,342.0 L112.3,343.7 L112.9,345.3 L113.4,350.3 L114.0,345.6 L114.5,345.0
 L115.1,344.8 L115.6,345.1 L116.2,345.8 L116.8,346.7 L117.3,347.8 L117.9,350.7 L118.4,350.7 L119.0,353.3
 L119.6,355.5 L120.1,355.1 L120.7,356.9 L121.2,357.7 L121.8,359.0 L122.4,360.4 L122.9,358.7 L123.5,357.8
 L124.0,360.2 L124.6,357.8 L125.1,358.5 L125.7,361.4 L126.3,360.5 L126.8,361.9 L127.4,363.4 L127.9,364.7
 L128.5,367.2 L129.1,367.6 L129.6,369.1 L130.2,370.4 L130.7,371.6 L131.3,372.9 L131.9,371.4 L132.4,369.8
 L133.0,369.0 L133.5,368.7 L134.1,368.8 L134.6,369.3 L135.2,370.0 L135.8,372.2 L136.3,372.0 L136.9,373.2
 L137.4,375.1 L138.0,376.6 L138.6,376.3 L139.1,377.6 L139.7,378.8 L140.2,379.6 L140.8,377.9 L141.4,379.2
 L141.9,375.1 L142.5,376.9 L143.0,377.7 L143.6,375.3 L144.1,376.2 L144.7,377.1 L145.3,378.1 L145.8,379.3
 L146.4,381.3 L146.9,383.0 L147.5,384.4 L148.1,383.4 L148.6,384.6 L149.2,385.9 L149.7,385.1 L150.3,382.7
 L150.9,383.9 L151.4,383.9 L152.0,383.9 L152.5,384.2 L153.1,384.7 L153.6,385.4 L154.2,386.2 L154.8,387.1
 L155.3,387.8 L155.9,385.7 L156.4,386.9 L157.0,388.1 L157.6,388.0 L158.1,388.6 L158.7,389.4 L159.2,386.8
 L159.8,388.0 L160.4,387.9 L160.9,384.4 L161.5,386.1 L162.0,387.4 L162.6,386.2 L163.1,387.1 L163.7,388.9
 L164.3,389.3 L164.8,390.8 L165.4,392.3 L165.9,393.5 L166.5,394.5 L167.1,395.3 L167.6,401.8 L168.2,392.6
 L168.7,393.3 L169.3,392.8 L169.9,392.4 L170.4,392.2 L171.0,392.4 L171.5,392.9 L172.1,390.9 L172.6,392.1
 L173.2,393.3 L173.8,393.4 L174.3,393.9 L174.9,394.4 L175.4,395.1 L176.0,395.6 L176.6,402.9 L177.1,393.9
 L177.7,391.8 L178.2,392.6 L178.8,390.2 L179.4,390.2 L179.9,391.3 L180.5,392.2 L181.0,393.1 L181.6,394.0
 L182.1,394.9 L182.7,393.9 L183.3,394.5 L183.8,395.0 L184.4,395.4 L184.9,396.1 L185.5,397.8 L186.1,395.5
 L186.6,396.0 L187.2,392.5 L187.7,392.0 L188.3,392.9 L188.9,393.5 L189.4,394.1 L190.0,393.5 L190.5,394.6
 L191.1,395.7 L191.6,396.5 L192.2,396.0 L192.8,396.4 L193.3,396.5 L193.9,397.4 L194.4,402.3 L195.0,396.5
 L195.6,396.8 L196.1,396.0 L196.7,392.8 L197.2,392.7 L197.8,393.0 L198.4,393.5 L198.9,394.4 L199.5,395.5
 L200.0,395.7 L200.6,396.4 L201.1,397.1 L201.7,397.7 L202.3,397.9 L202.8,398.6 L203.4,405.9 L203.9,398.5
 L204.5,396.4 L205.1,395.2 L205.6,394.6 L206.2,394.4 L206.7,394.9 L207.3,395.0 L207.9,395.8 L208.4,396.6
 L209.0,397.5 

### Quantitative comparison

#### Assert that the deviance is within limits

First, create some error plots, as the simulations should use the same fixed timestep this is possible.

In [ ]:
List<Plot2Ddata> dataCL_Err = new List<Plot2Ddata>();
List<Plot2Ddata> dataEvap_Err = new List<Plot2Ddata>();

for(int i = 0; i < dataCL.Count(); i++){
    Plot2Ddata errData = new Plot2Ddata();
    for(int j = 0; j < dataCL[i].dataGroups.Count() & j < 4; j++){
        var group = dataCL[i].dataGroups[j];
        var group_ref = dataCL_Ref[i].dataGroups[j];
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        for(int k = 0; k< val.Values.Count(); k++){
            val.Values[k] = group_ref.Values[k] - group.Values[k];
        }
        val.Format = group.Format;
        errData.dataGroups = errData.dataGroups.Cat(val);
    }
    dataCL_Err.Add(errData);
}

for(int i = 0; i < dataEvap.Count(); i++){
    Plot2Ddata errData = new Plot2Ddata();
    for(int j = 0; j < dataEvap[i].dataGroups.Count() & j < 4; j++){
        var group = dataEvap[i].dataGroups[j];
        var group_ref = dataEvap_Ref[i].dataGroups[j];
        Plot2Ddata.XYvalues val = new Plot2Ddata.XYvalues(group.Name, group.Abscissas, group.Values);
        for(int k = 0; k< val.Values.Count(); k++){
            val.Values[k] = group_ref.Values[k] - group.Values[k];
        }
        val.Format = group.Format;
        errData.dataGroups = errData.dataGroups.Cat(val);
    }
    dataEvap_Err.Add(errData);
}

if(dataEvap[0].dataGroups[0].Abscissas.Count() != dataEvap_Ref[0].dataGroups[0].Abscissas.Count() | dataCL[0].dataGroups[0].Abscissas.Count() != dataCL_Ref[0].dataGroups[0].Abscissas.Count()){
    Console.WriteLine("\x1b[31mError: datasets of different lengths!\x1b[0m");
    Errors++;
}

In [ ]:
var gp = new Gnuplot();
gp.SetMultiplot(2,2);

Using gnuplot: C:\Program Files (x86)\FDY\BoSSS\BoSSS\bin\native\win\gnuplot-gp510-20160418-win32-mingw\gnuplot\bin\gnuplot.exe


In [ ]:
try{
    gp.SetSubPlot(0, 0);

    data = dataCL_Err.Pick(4);
    data.Xlabel = "time [s]";
    data.Ylabel = "angle [°]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact Angle";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(0, 1);

    data = dataEvap_Err.Pick(3);
    data.Xlabel = "time [s]";
    data.Ylabel = "Total Heat Flux [W]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Heatflux";
    data.ShowLegend = true;
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(1, 0);

    data = dataCL_Err.Pick(1);
    data.Xlabel = "time [s]";
    data.Ylabel = "Pos Y [m]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Position";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);

    gp.SetSubPlot(1, 1);

    data = dataCL_Err.Pick(3);
    data.Xlabel = "time [s]";
    data.Ylabel = "V_y [m/s]";
    data.XrangeMin = 0;
    data.XrangeMax = 12;
    data.LabelTitleFont = 16;
    data.TitleFont = 24;
    data.Title = "Contact-line Velocity Reference";
    data.LegendAlignment = new string[] {"i","l","t"};

    data.ToGnuplot(gp);
} catch {
    Console.WriteLine("\x1b[31mError: plotting data!\x1b[0m");
    Errors++;
}

set key font ",16"inside left top Left reverse 
set key font ",16"inside left top Left reverse 
set key font ",16"inside left top Left reverse 
set key font ",16"inside left top Left reverse 


In [ ]:
gp.PlotSVG(1600, 1200)

<?xml version="1.0" encoding="utf-8" standalone="no"?>
<!DOCTYPE svg PUBLIC "-//W3C//DTD SVG 1.1//EN"
 "http://www.w3.org/Graphics/SVG/1.1/DTD/svg11.dtd">
 

 Gnuplot 
 Produced by GNUPLOT 5.1 patchlevel 0 

 

 
 

 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 -6x10 -5 
 
 
 
 
 -5x10 -5 
 
 
 
 
 -4x10 -5 
 
 
 
 
 -3x10 -5 
 
 
 
 
 -2x10 -5 
 
 
 
 
 -1x10 -5 
 
 
 
 
 0 
 
 
 
 
 1x10 -5 
 
 
 
 
 2x10 -5 
 
 
 
 
 0 
 
 
 
 
 2 
 
 
 
 
 4 
 
 
 
 
 6 
 
 
 
 
 8 
 
 
 
 
 10 
 
 
 
 
 12 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 
 angle [°] 
 
 
 
 
 time [s] 
 
 
 
 
 Contact Angle 
 
 
 q:0 
 
 
 
 
 q:0 
 
 
 
	<path stroke='rgb( 0, 0, 0)' d='M115.8,117.1 L169.2,117.1 M96.4,206.2 L96.9,206.2 L97.5,469.9 L98.0,518.7 L98.6,470.8 L99.1,437.4
 L99.6,410.7 L100.2,387.7 L100.7,369.7 L101.2,355.2 L101.8,344.5 L102.3,314.9 L102.9,314.4 L103.4,310.4
 L103.9,310.7 L104.5,311.6 L105.0,311.7 L105.5,311.0 L106.1,309.0 L106.6,306.8 L107.2,295.7 L107.7,294.6
 L108.2,296.0 L108.8,298.4 L109.3,301.5 L109.9,305.5 L110.4,309.7 L110.9,254.9 L111.5,249.4 L112.0,249.7
 L112.5,247.3 L113.1,243.3 L113.6,242.1 L114.2,241.0 L114.7,239.6 L115.2,238.7 L115.8,238.0 L116.3,237.5
 L116.8,237.4 L117.4,237.7 L117.9,238.8 L118.5,253.7 L119.0,251.2 L119.5,194.6 L120.1,206.6 L120.6,206.5
 L121.2,209.5 L121.7,212.5 L122.2,214.8 L122.8,216.6 L123.3,217.9 L123.8,219.1 L124.4,220.2 L124.9,220.8
 L125.5,232.8 L126.0,234.0 L126.5,230.4 L127.1,223.6 L127.6,220.8 L128.1,170.0 L128.7,179.9 L129.2,189.9
 L129.8,189.0 L130.3,191.6 L130.8,194.3 L131.4,196.5 L131.9,198.2 L132.5,199.3 L133.0,200.1 L133.5,200.3
 L134.1,200.0 L134.6,217.2 L135.1,217.5 L135.7,217.3 L136.2,217.4 L136.8,167.2 L137.3,170.2 L137.8,178.3
 L138.4,185.4 L138.9,191.3 L139.4,196.3 L140.0,200.8 L140.5,203.9 L141.1,199.2 L141.6,208.1 L142.1,201.4
 L142.7,198.7 L143.2,208.6 L143.8,200.0 L144.3,207.6 L144.8,207.7 L145.4,208.7 L145.9,165.4 L146.4,174.7
 L147.0,174.1 L147.5,188.1 L148.1,192.8 L148.6,189.3 L149.1,200.4 L149.7,203.2 L150.2,205.6 L150.7,207.6
 L151.3,200.6 L151.8,207.8 L152.4,208.2 L152.9,207.7 L153.4,207.7 L154.0,208.4 L154.5,159.3 L155.1,169.4
 L155.6,177.7 L156.1,184.5 L156.7,190.2 L157.2,194.9 L157.7,198.4 L158.3,195.0 L158.8,204.0 L159.4,206.0
 L159.9,199.8 L160.4,197.3 L161.0,206.6 L161.5,198.8 L162.0,195.8 L162.6,194.6 L163.1,161.4 L163.7,163.3
 L164.2,180.4 L164.7,180.5 L165.3,183.7 L165.8,197.3 L166.4,201.1 L166.9,203.6 L167.4,206.1 L168.0,207.9
 L168.5,203.4 L169.0,202.0 L169.6,201.5 L170.1,211.1 L170.7,206.0 L171.2,205.3 L171.7,171.3 L172.3,169.9
 L172.8,172.8 L173.3,179.5 L173.9,185.5 L174.4,190.1 L175.0,193.6 L175.5,196.0 L176.0,197.7 L176.6,198.5
 L177.1,198.4 L177.7,211.2 L178.2,205.6 L178.7,203.7 L179.3,212.9 L179.8,214.1 L180.3,169.7 L180.9,168.4
 L181.4,171.6 L182.0,178.8 L182.5,195.0 L183.0,195.6 L183.6,198.2 L184.1,207.0 L184.6,209.6 L185.2,207.3
 L185.7,212.5 L186.3,208.3 L186.8,206.8 L187.3,206.3 L187.9,206.5 L188.4,207.3 L189.0,156.8 L189.5,166.3
 L190.0,170.3 L190.6,178.5 L191.1,186.0 L191.6,192.2 L192.2,196.9 L192.7,200.4 L193.3,209.8 L193.8,207.9
 L194.3,207.8 L194.9,213.2 L195.4,213.4 L195.9,208.6 L196.5,207.4 L197.0,207.8 L197.6,160.6 L198.1,168.8
 L198.6,180.8 L199.2,183.6 L199.7,196.4 L200.3,201.9 L200.8,202.6 L201.3,205.2 L201.9,207.4 L202.4,209.0
 L202.9,210.0 L203.5,216.7 L204.0,217.0 L204.6,217.7 L205.1,213.3 L205.6,212.4 L206.2,174.3 L206.7,171.6
 L207.2,176.3 L207.8,192.8 L208.3,199.7 L208.9,201.5 L209.4,205.1 L209.9,208.3 L210.5,214.1 L211.0,212.6
 L211.5,212.7 L212.1,212.3 L212.6,218.3 L213.2,218.4 L213.7,214.0 L214.2,220.9 L214.8,169.3 L215.3,172.9
 L215.9,177.4 L216.4,186.2 L216.9,201.0 L217.5,206.2 L218.0,210.3 L218.5,213.4 L219.1,213.2 L219.6,214.3
 L220.2,219.4 L220.7,220.1 L221.2,220.7 L221.8,220.9 L222.3,216.9 L222.8,215.8 L223.4,164.3 L223.9,172.3
 L224.5,185.2 L225.0,194.2 L225.5,201.2 L226.1,206

#### Calculate relative Error Norm
$ \varepsilon_{abs} = ||u - u_{ref}|| = (\sum (u_i - u_{i,ref})^2)^{\frac{1}{2}} $  

$ ||u_{ref}|| = (\sum (u_{i,ref})^2)^{\frac{1}{2}} $  

$ \varepsilon_{rel} = \frac{||u - u_{ref}||}{||u_{ref}||} $  

In [ ]:
int N_end = 1200; // endtime 12 a dt 0.01
Dictionary<string, List<double>> errCL_abs      = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> errEvap_abs    = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> normCL_ref     = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> normEvap_ref   = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> errCL_rel      = new Dictionary<string, List<double>>();
Dictionary<string, List<double>> errEvap_rel    = new Dictionary<string, List<double>>();

In [ ]:
for(int i = 0; i < dataCL_Ref.Count(); i++){
    for(int j = 0; j < dataCL_Ref[i].dataGroups.Count() & j < 4; j++){
        string name = dataCL_Ref[i].dataGroups[j].Name;
        var vals_ref = dataCL_Ref[i].dataGroups[j].Values;
        var vals = dataCL[i].dataGroups[j].Values;
        double norm = 0.0, err = 0.0;
        for(int k = 0; k< N_end; k++){
            err += Math.Pow(vals_ref[k] - vals[k], 2.0);
            norm += Math.Pow(vals_ref[k], 2.0);
        }
        err = Math.Sqrt(err);
        norm = Math.Sqrt(norm);

        if (!errCL_abs.ContainsKey(name)){
            errCL_abs[name] = new List<double>();
            normCL_ref[name] = new List<double>();
            errCL_rel[name] = new List<double>();
        }
        errCL_abs[name].Add(err);
        normCL_ref[name].Add(norm);
        errCL_rel[name].Add(err/norm);
    }
}

for(int i = 0; i < dataEvap_Ref.Count(); i++){
    for(int j = 0; j < dataEvap_Ref[i].dataGroups.Count() & j < 4; j++){
        string name = dataEvap_Ref[i].dataGroups[j].Name;
        var vals_ref = dataEvap_Ref[i].dataGroups[j].Values;
        var vals = dataEvap[i].dataGroups[j].Values;
        double norm = 0.0, err = 0.0;
        for(int k = 0; k< N_end; k++){
            err += Math.Pow(vals_ref[k] - vals[k], 2.0);
            norm += Math.Pow(vals_ref[k], 2.0);
        }
        err = Math.Sqrt(err);
        norm = Math.Max(Math.Sqrt(norm), 1e-10); // bound very small norms!
        
        if (!errEvap_abs.ContainsKey(name)){
            errEvap_abs[name] = new List<double>();
            normEvap_ref[name] = new List<double>();
            errEvap_rel[name] = new List<double>();
        }
        errEvap_abs[name].Add(err);
        normEvap_ref[name].Add(norm);
        errEvap_rel[name].Add(err/norm);
    }
}

Set threshhold for relative errors and assert its been reached

In [ ]:
double thrsh = 0.01;

int[] cl_props = new int[] {1, 3, 4};
foreach(var rel_errs in errCL_rel){
    foreach(var index in cl_props){
        var err = rel_errs.Value[index];
        NUnit.Framework.Assert.IsTrue(err <= thrsh, $"Assertion failed for prop {index} of cl props in {rel_errs.Key}!");
        if(err > thrsh){
            Console.WriteLine($"\x1b[31mAssertion failed in {rel_errs.Key}!\x1b[0m");
            Errors++;
        }
    }
}

int[] evap_props = new int[] {3};
foreach(var rel_errs in errEvap_rel){
    foreach(var index in evap_props){
        var err = rel_errs.Value[index];
        NUnit.Framework.Assert.IsTrue(err <= thrsh, $"Assertion failed for prop {index} of evap props in {rel_errs.Key}!");
        if(err > thrsh){
            Console.WriteLine($"\x1b[31mAssertion failed in {rel_errs.Key}!\x1b[0m");
            Errors++;
        }
    }
}

#### Print out the result

In [ ]:
if(Errors != 0){
    Console.WriteLine($"\x1b[31mVerification failed with {Errors} errors!\x1b[0m");
}else{
    Console.WriteLine($"\x1b[32mVerification successfull!\x1b[0m");
}